# PulsimCore High-Performance API

This notebook demonstrates the new API features:

1. Standalone device objects with CRTP architecture
2. Advanced convergence aids (Gmin stepping, source stepping, pseudo-transient)
3. BDF integration with adaptive timestep control
4. Analytical solutions for validation
5. SIMD detection and high-performance features
6. Benchmark framework

In [ ]:
import pulsim as v2
import numpy as np
import matplotlib.pyplot as plt

print(f"API loaded successfully")

## 1. Device Objects

The API provides standalone device objects that can be configured independently.

In [ ]:
# Create passive devices
r1 = v2.Resistor(1000.0, "R1")  # 1k Ohm
c1 = v2.Capacitor(1e-6, 0.0, "C1")  # 1uF, 0V initial
l1 = v2.Inductor(1e-3, 0.0, "L1")  # 1mH, 0A initial

# Create sources
vs = v2.VoltageSource(12.0, "Vdc")
cs = v2.CurrentSource(0.001, "I1")

print(f"R1: {r1.resistance()} Ohms")
print(f"C1: {c1.capacitance() * 1e6:.2f} uF")
print(f"L1: {l1.inductance() * 1e3:.2f} mH")
print(f"Vdc: {vs.voltage()} V")
print(f"I1: {cs.current() * 1e3:.2f} mA")

## 2. Device Types and Solver Status

The API provides comprehensive enumerations.

In [ ]:
# Device types
print("Device Types:")
for dtype in [v2.DeviceType.Resistor, v2.DeviceType.Capacitor, 
              v2.DeviceType.Inductor, v2.DeviceType.MOSFET]:
    print(f"  - {dtype}")

print("\nSolver Status Codes:")
for status in [v2.SolverStatus.Success, v2.SolverStatus.MaxIterationsReached,
               v2.SolverStatus.SingularMatrix, v2.SolverStatus.Diverging]:
    print(f"  - {v2.solver_status_to_string(status)}")

## 3. Solver Configuration

The API provides fine-grained control over solver behavior.

In [ ]:
# Default tolerances
tols = v2.Tolerances.defaults()
print("Default Tolerances:")
print(f"  Voltage absolute: {tols.voltage_abstol:.2e}")
print(f"  Current absolute: {tols.current_abstol:.2e}")
print(f"  Residual: {tols.residual_tol:.2e}")

# Newton solver options
opts = v2.NewtonOptions()
print(f"\nNewton Options:")
print(f"  Max iterations: {opts.max_iterations}")
print(f"  Initial damping: {opts.initial_damping}")
print(f"  Auto damping: {opts.auto_damping}")

## 4. DC Convergence Aids

The API includes advanced strategies for difficult DC operating points.

In [ ]:
# Gmin stepping configuration
gmin = v2.GminConfig()
print("Gmin Stepping:")
print(f"  Initial Gmin: {gmin.initial_gmin:.2e}")
print(f"  Final Gmin: {gmin.final_gmin:.2e}")
print(f"  Reduction factor: {gmin.reduction_factor}")
print(f"  Required steps: {gmin.required_steps()}")

# Source stepping configuration
src = v2.SourceSteppingConfig()
print(f"\nSource Stepping:")
print(f"  Initial scale: {src.initial_scale}")
print(f"  Final scale: {src.final_scale}")
print(f"  Max steps: {src.max_steps}")

# Pseudo-transient configuration
ptran = v2.PseudoTransientConfig()
print(f"\nPseudo-Transient:")
print(f"  Initial dt: {ptran.initial_dt:.2e}")
print(f"  Max dt: {ptran.max_dt:.2e}")

# DC convergence with auto strategy
dc_config = v2.DCConvergenceConfig()
print(f"\nDC Strategy: {dc_config.strategy}")

## 5. Integration Methods

BDF order control and adaptive timestep with PI controller.

In [ ]:
# BDF order configuration
bdf = v2.BDFOrderConfig()
print("BDF Order Control:")
print(f"  Min order: {bdf.min_order}")
print(f"  Max order: {bdf.max_order}")
print(f"  Initial order: {bdf.initial_order}")
print(f"  Auto order: {bdf.enable_auto_order}")

# Timestep configurations
print("\nTimestep Configurations:")
for name, cfg in [("Default", v2.TimestepConfig.defaults()),
                   ("Conservative", v2.TimestepConfig.conservative()),
                   ("Aggressive", v2.TimestepConfig.aggressive())]:
    print(f"  {name}:")
    print(f"    dt_min: {cfg.dt_min:.2e}, dt_max: {cfg.dt_max:.2e}")
    print(f"    safety_factor: {cfg.safety_factor:.2f}")

## 6. Analytical Solutions for Validation

Built-in analytical solutions for RC, RL, and RLC circuits.

In [ ]:
# RC circuit: 1k Ohm, 1uF, step from 0V to 5V
rc = v2.RCAnalytical(1000, 1e-6, 0.0, 5.0)
tau = rc.tau()

print(f"RC Circuit (R=1k, C=1uF):")
print(f"  Time constant: {tau * 1e3:.3f} ms")
print(f"  V(0): {rc.voltage(0):.3f} V")
print(f"  V(tau): {rc.voltage(tau):.3f} V (63.2%)")
print(f"  V(5*tau): {rc.voltage(5*tau):.3f} V (99.3%)")

# Plot RC response
t = np.linspace(0, 5*tau, 100)
v = [rc.voltage(ti) for ti in t]

plt.figure(figsize=(10, 5))
plt.plot(t * 1e3, v, 'b-', linewidth=2)
plt.axhline(y=5*0.632, color='gray', linestyle='--', alpha=0.5)
plt.axvline(x=tau*1e3, color='gray', linestyle='--', alpha=0.5)
plt.xlabel('Time (ms)')
plt.ylabel('Voltage (V)')
plt.title('RC Circuit Step Response')
plt.grid(True, alpha=0.3)
plt.annotate(f'τ = {tau*1e3:.2f} ms', xy=(tau*1e3, 5*0.632), 
             xytext=(tau*1e3 + 0.5, 2.5), fontsize=10)
plt.show()

In [ ]:
# RL circuit: 1k Ohm, 1mH, 10V source
rl = v2.RLAnalytical(1000, 1e-3, 10.0, 0.0)
tau_rl = rl.tau()

print(f"RL Circuit (R=1k, L=1mH, V=10V):")
print(f"  Time constant: {tau_rl * 1e6:.3f} us")
print(f"  Final current: {rl.I_final() * 1e3:.3f} mA")

# Plot RL response
t = np.linspace(0, 5*tau_rl, 100)
i = [rl.current(ti) * 1e3 for ti in t]

plt.figure(figsize=(10, 5))
plt.plot(t * 1e6, i, 'r-', linewidth=2)
plt.xlabel('Time (us)')
plt.ylabel('Current (mA)')
plt.title('RL Circuit Step Response')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# RLC circuits with different damping
print("RLC Circuit Damping Analysis:\n")

# Underdamped: small R
rlc_under = v2.RLCAnalytical(10, 1e-3, 1e-6, 10.0, 0.0, 0.0)
print(f"Underdamped (R=10):")
print(f"  omega_0: {rlc_under.omega_0():.1f} rad/s")
print(f"  zeta: {rlc_under.zeta():.3f}")
print(f"  Damping: {rlc_under.damping_type()}")

# Overdamped: large R
rlc_over = v2.RLCAnalytical(1000, 1e-3, 1e-6, 10.0, 0.0, 0.0)
print(f"\nOverdamped (R=1000):")
print(f"  omega_0: {rlc_over.omega_0():.1f} rad/s")
print(f"  zeta: {rlc_over.zeta():.3f}")
print(f"  Damping: {rlc_over.damping_type()}")

# Plot underdamped response
t = np.linspace(0, 0.01, 500)
v_under = [rlc_under.voltage(ti) for ti in t]

plt.figure(figsize=(10, 5))
plt.plot(t * 1e3, v_under, 'g-', linewidth=2)
plt.xlabel('Time (ms)')
plt.ylabel('Voltage (V)')
plt.title('Underdamped RLC Step Response')
plt.grid(True, alpha=0.3)
plt.show()

## 7. Waveform Validation

Compare simulation results against analytical solutions.

In [ ]:
# Generate analytical waveform
rc = v2.RCAnalytical(1000, 1e-6, 0.0, 5.0)
times = np.linspace(0, 5e-3, 50)

# Simulated data (with slight error for demo)
sim_data = [(t, rc.voltage(t) + np.random.normal(0, 0.001)) for t in times]
ana_data = [(t, rc.voltage(t)) for t in times]

# Compare waveforms
result = v2.compare_waveforms("RC_validation", sim_data, ana_data, 0.01)

print("Validation Result:")
print(f"  Test: {result.test_name}")
print(f"  Passed: {result.passed}")
print(f"  Max error: {result.max_error:.6f}")
print(f"  RMS error: {result.rms_error:.6f}")
print(f"  Points: {result.num_points}")

In [ ]:
# Export validation results
csv_output = v2.export_validation_csv([result])
print("CSV Export:")
print(csv_output)

json_output = v2.export_validation_json([result])
print("\nJSON Export:")
print(json_output)

## 8. High-Performance Features

SIMD detection and linear solver configuration.

In [ ]:
# SIMD detection
simd_level = v2.detect_simd_level()
vector_width = v2.simd_vector_width()

print("SIMD Capability:")
print(f"  Level: {simd_level}")
print(f"  Vector width: {vector_width} doubles")

# Linear solver configuration
ls_config = v2.LinearSolverConfig()
print(f"\nLinear Solver:")
print(f"  Pivot tolerance: {ls_config.pivot_tolerance:.2e}")
print(f"  Reuse symbolic: {ls_config.reuse_symbolic}")
print(f"  Deterministic pivoting: {ls_config.deterministic_pivoting}")

## 9. Benchmark Framework

Tools for performance measurement and comparison.

In [ ]:
# Create benchmark structures
timing = v2.BenchmarkTiming()
timing.name = "RC_transient"
timing.iterations = 1000

result = v2.BenchmarkResult()
result.circuit_name = "RC_filter"
result.num_nodes = 2
result.num_devices = 3
result.num_timesteps = 5000

print("Benchmark Result:")
print(f"  Circuit: {result.circuit_name}")
print(f"  Nodes: {result.num_nodes}")
print(f"  Devices: {result.num_devices}")
print(f"  Timesteps: {result.num_timesteps}")

## Summary

The API provides:

- **Standalone device objects** with CRTP architecture for 2x+ performance
- **Advanced convergence aids** for difficult DC operating points
- **BDF integration** with automatic order control (1-2)
- **PI timestep controller** for adaptive simulation
- **Built-in validation** against analytical solutions
- **SIMD optimization** with runtime detection
- **Benchmark framework** for performance measurement

See the [Migration Guide](../../docs/migration-v1-to-v2.md) for transitioning from v1.